In [43]:
import sqlite3
import pandas as pd
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '')
ds_price['STOCK_CLOSE'] = pd.to_numeric(ds_price['STOCK_CLOSE'])
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['EP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['DECILE'] = (ds_ratio.groupby('STOCK_YEAR')['EP']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
s1 = ds_ratio.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

print("5--- %s seconds ---" % (time.time() - st))
st = time.time()

s1.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')



#ds_price

1--- 3.209183692932129 seconds ---
2--- 0.45102572441101074 seconds ---
3--- 4.604263067245483 seconds ---
3--- 3.8922228813171387 seconds ---
4--- 1.431082010269165 seconds ---
5--- 0.0390019416809082 seconds ---


STOCK_YEAR,2011,2012,2013,2014,2015,2016,2017
DECILE,,,,,,,
0.0,0.837704,1.115118,0.995606,0.925045,0.885809,1.107489,0.926600
1.0,0.920135,1.084825,1.175648,1.014271,1.094995,1.135392,0.887672
2.0,0.857687,1.184520,0.969740,1.067192,1.039543,1.164843,0.903127
3.0,0.952850,1.267875,0.991406,1.126042,1.083746,1.091764,0.938852
4.0,1.043203,1.271877,1.062791,1.098411,1.080078,1.151107,0.940865
5.0,1.119233,1.301727,1.098700,1.146277,1.141121,1.275363,0.969936
6.0,1.043732,1.284205,1.106619,1.245019,1.146367,1.149280,0.946592
7.0,1.119494,1.407103,1.096132,1.144480,1.094093,1.042661,0.959581
8.0,1.181776,1.397383,1.231342,1.302798,1.205132,1.103859,0.959370
